#Latihan CFG - Rachmansyah Adhi Widhianto (1301170066)

#Create Rules Manually
Berikut adalah contoh production rules yang dibuat manual dalam format string biasa.
```
cfg_rules = """
S -> NP VP
NP -> Det N | PropN
Det -> PosPro | Art
VP -> Vt NP
Art -> 'the' | 'a'
PropN -> 'Alice'
N -> 'duck' | 'telescope' | 'park'
Vt -> 'saw'
PosPro -> 'my' | 'her'
"""
```

#Create CFG from String
Selanjutnya buat objek grammar CFG dari string di atas. Akan terbentuk objek grammar dengan 15 productions.
```
cfg = nltk.CFG.fromstring(cfg_rules)
print(cfg)
```
#Parsing
Dari grammar tersebut, kita coba untuk mendapatkan parse tree dari sebuah kalimat. Tampilkan hasil parse tree.
```
from nltk.parse.chart import BottomUpChartParser
parser = BottomUpChartParser(cfg)
sentence = 'the duck saw Alice'.split()
parsed = list(parser.parse(sentence))
print(parsed[0])
parsed[0].draw()
```

#Extract Production Rules From Corpus
Sebagai alternatif cara membuat grammar selain manual, kita bisa mengekstrak rules dari corpus yang telah dianotasi. Corpus berbahasa Inggris yang terkenal misalnya penn treebank. Salah satu corpus treebank berbahasa indonesia bisa dilihat di https://github.com/famrashel/idn-treebank.
```
from nltk import Tree

parsed_string = "(S      (NP-SBJ-1    (NN      Kisah) (NP      (NN      pendiri) (NNP      Facebook))) (VP      (VB      dibuat) (S  (NP-SBJ  *-1) (NP-PRD  (NN      komik)))))"
t = Tree.fromstring(parsed_string)
print(t.productions())
```

#Create CFG from Production Rules
Setelah mendapatkan list production rules dari corpus, selanjutnya kita bisa membuat grammar CFG. Grammar ini selanjutnya bisa dipakai untuk melakukan parsing pada data testing atau data baru.
```
from nltk.grammar import CFG, Nonterminal
grammar = CFG(Nonterminal('S'), t.productions())
print(grammar)
```

#Menambahkan Production Rules ke existing grammar
Jika suatu saat corpus kita bertambah, maka kita bisa menambahkan production rules baru ke existing grammar.
```
from nltk.grammar import Nonterminal, Production

#Cara membuat objek Nonterminal
lhs = Nonterminal('S')
rhs1 = Nonterminal('A')
rhs2 = Nonterminal('B')
#Untuk Objek terminal cukup string saja
rhs3 = "saya"
#Buat objek Production
newprod = Production(lhs,[rhs1,rhs2,rhs3])

#Tambahkan ke existing grammar
grammar.productions().append(newprod)
print(grammar)
```

#Tugas
1. Ekstraklah Production Rules dari minimal 100 kalimat pertama (tergantung kemampuan perangkat) dari Corpus https://github.com/famrashel/idn-treebank/blob/master/Indonesian_Treebank.bracket. Anda mungkin perlu menghubungkan gdrive ke colab. Cek tutorial ini :  https://www.marktechpost.com/2019/06/07/how-to-connect-google-colab-with-google-drive/

2. Print top-10 Rules yang paling sering muncul. Hint gunakan collection.Counter.

3. Buatlah grammar CFG dari production rules yang telah diekstrak.
4. Lakukan Parsing terhadap kalimat berikut ini:
```
Binatang ini tidak bisa dibunuh karena masyarakat India menganggap mereka suci .
```

5. Print top-50 lexicon atau terminal symbol.

dari contoh ini: (S  (PP  (IN	  (Selama)) (NP  (CD	  (bertahun-tahun)))) (NP-SBJ  (NN	  (monyet))) (VP  (VB	  (mengganggu)) (NP  (NN	  (warga)) (NNP	  (Delhi)))) (Z	  (.)))
yang dimaksuk lexicon adalah : Selama, bertahun-tahun, monyet, mengganggu, warga, Delhi, .

6. Buatlah 5 kalimat baru dari top-50 lexicon, lalu parsing kalimat tersebut.



# JAWABAN

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
#contoh path
path = '/content/drive/My Drive/Colab Notebooks/NLP - Parsing/idn-treebank-master/'

In [0]:
import nltk
from nltk import Tree
import re
from collections import Counter
from nltk.grammar import CFG, Nonterminal
from nltk.parse.chart import BottomUpChartParser

###Ekstrak Production Rules (15 poin)

In [13]:
tree = [] # List untuk menampung tree
rules = [] # List untuk menampung production rules

n = 200 # Banyak tree yang diambil dari corpus
pertama = ''
with open(path + 'Indonesian_Treebank.bracket') as f: # Membuka file corpus1111
    gen = (x for x in f if len(tree) < n) # Generator untuk kondisi loop 
    for line in gen:
      line = re.sub(r"\(([^\(]+)\)", r"\1", line) # Pre-processing untuk mencocokan format tree dengan parser
      if pertama == '':
        pertama = line
      tree.append(Tree.fromstring(line)) # Memasukkan tree ke list
      rules += Tree.fromstring(line).productions() # Memasukkan production rules dari tree ke list

print(pertama)

(NP  (NN	  Kera) (SBAR  (SC	  untuk) (S  (NP-SBJ  *) (VP  (VB	  amankan) (NP  (NN	  pesta olahraga))))))



###Print top-10 rules (15 poin)

In [14]:
c = Counter(rules) # Menghitung rules dengan counter
c.most_common(10) # Menampilkan top 10 rules

[(PP -> IN NP, 283),
 (S -> NP-SBJ VP, 199),
 (Z -> '.', 192),
 (SBAR -> SC S, 160),
 (NP-SBJ -> '*', 154),
 (VP -> VB NP, 143),
 (Z -> ',', 134),
 (NP -> NN, 120),
 (SC -> 'yang', 99),
 (VP -> MD VP, 85)]

###Buat Grammar CFG (20 poin)

In [15]:
grammar = CFG(Nonterminal('S'), rules) # Mengubah production rules menjadi Grammar
grammar

<Grammar with 7705 productions>

###Lakukan parsing pada contoh kalimat (20 poin)

In [0]:
s = "Binatang ini tidak bisa dibunuh karena masyarakat India menganggap mereka suci ."
parser = BottomUpChartParser(grammar) # Membuat parser dari grammar
sentence = s.split() # Memisahkan kalimat berdasarkan spasi
parsed = list(parser.parse(sentence)) # Hasil parsing dari kalimat
print(parsed[0].pretty_print()) # Menampilkan salah satu hasil parsing
# parsed[0].draw()

                                           S                                                            
                                     ______|____________________________________________________         
                                  NP-SBJ                                                        |       
                   _________________|________________________                                   |        
                  NP                                         PP                                 |       
    ______________|_____                    _________________|______                            |        
   |                  S-NOM                |                      S-NOM                       NP-PRD    
   |        ____________|____              |                  ______|_______                ____|_____   
   |       |                 VP            |               NP-SBJ           |              NP         | 
   |       |             ____|______       |       

###Print top-50 Lexicon (10 poin)

In [0]:
lexicon = [] # List untuk menampung lexicon
for i in tree:
  lexicon += i.leaves() # Menambahkan setiap leaves dari tree ke lexicon

c = Counter(lexicon) # Menghitung lexicon dengan counter
c.most_common(50) # Menampilkan top 50 lexicon

[('.', 241),
 ('*', 236),
 (',', 189),
 ('yang', 121),
 ('*-1', 113),
 ('dan', 96),
 ('di', 94),
 ('nya', 80),
 ('0', 79),
 ('"', 70),
 ('untuk', 64),
 ('akan', 50),
 ('dari', 49),
 ('itu', 42),
 ('tahun', 42),
 ('ini', 41),
 ('*-2', 41),
 ('dengan', 40),
 ('*U*', 33),
 ('*T*-1', 30),
 ('dalam', 30),
 ('mengatakan', 29),
 ('pada', 29),
 ('Rp', 28),
 ('menjadi', 23),
 ('lebih', 22),
 ('mereka', 20),
 ('tidak', 20),
 ('orang', 20),
 ('kata', 19),
 ('dia', 19),
 ('tersebut', 18),
 ('oleh', 18),
 ('-', 18),
 ('ke', 17),
 ('persen', 17),
 ('Indonesia', 17),
 ('banyak', 16),
 ('sebagai', 15),
 ('2007', 15),
 ('triliun', 15),
 ('sebesar', 15),
 ('kepada', 14),
 ('miliar', 14),
 ('pesawat', 14),
 ('bisa', 13),
 ('telah', 13),
 ('satu', 13),
 ('2006', 13),
 ('monyet', 12)]

###Buat 5 kalimat baru (manual) dari top-50 lexicon, dan Parsing (20 poin)

In [0]:
s = [
     "monyet menjadi pesawat .",
     "pesawat tersebut mengatakan yang tidak bisa .",
     "banyak monyet menjadi lebih dari satu .",
     "mereka mengatakan akan menjadi pesawat dari tahun ini .",
     "orang dari Indonesia menjadi lebih banyak .",
] # Kalimat baru (manual)

for sentence in s:
  print(sentence) # Menampilkan kalimat yang akan di parse
  sentence = sentence.split() # Memisahkan kalimat berdasarkan spasi
  parsed = list(parser.parse(sentence)) # Hasil parsing dari kalimat
  print(parsed[0].pretty_print()) # Menampilkan salah satu hasil parsing
  print()
  print("--------------------------------------------------------------------")
  print()

monyet menjadi pesawat .
                      S                    
          ____________|____________         
       S-SBJ                     NP-PRD    
   ______|______             ______|_____   
NP-SBJ          VP          NP           NP
  |             |           |            |  
  NN            VB          NN          NNP
  |             |           |            |  
monyet       menjadi     pesawat         . 

None

--------------------------------------------------------------------

pesawat tersebut mengatakan yang tidak bisa .
                              S                         
    __________________________|_____                     
   |        |                       VP                  
   |        |          _____________|____                
   |        |         |                 SBAR            
   |        |         |        __________|____           
   |        |         |       |              SINV       
   |        |         |       |           ____|___